# Project: Kidney Disease Dataset
## Importing Necessary Libraries
- **Purpose:** To load essential Python libraries for data handling, preprocessing, and statistical analysis.

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import seaborn as sns
from matplotlib import pyplot
from scipy.stats import norm, ttest_ind, ttest_rel, f_oneway
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.preprocessing import LabelEncoder

## Loading the Dataset
- **Purpose:** To read the kidney disease dataset into a pandas DataFrame for analysis.

In [2]:
dataset = pd.read_csv('kidney_disease.csv')
dataset

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


## Exploratory Data Analysis (EDA)
## Checking Missing Values
- **Purpose:** Identify the columns with missing values for imputation.

In [3]:
dataset.isna().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [5]:
dataset.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

## Categorizing Numerical and Categorical Columns
- **Purpose:** Separate numerical and categorical columns for targeted preprocessing.

In [6]:
def quanqual(dataset):
    quan = []
    qual = []
    for ColumnName in dataset.columns:
        if dataset[ColumnName].dtype == 'O':  
            qual.append(ColumnName)
        else:
            quan.append(ColumnName) 
    return quan,qual

In [7]:
quan, qual=quanqual(dataset)

In [8]:
quan

['id', 'age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo']

In [9]:
qual

['rbc',
 'pc',
 'pcc',
 'ba',
 'pcv',
 'wc',
 'rc',
 'htn',
 'dm',
 'cad',
 'appet',
 'pe',
 'ane',
 'classification']

## Handling Missing Values
## Imputing Missing Values
- **Purpose:** Replace missing numerical values with mean and categorical values with the most frequent.

In [10]:
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

In [11]:
dataset[quan] = numerical_imputer.fit_transform(dataset[quan])
dataset[qual] = categorical_imputer.fit_transform(dataset[qual])

In [12]:
dataset[quan]

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
0,0.0,48.0,80.0,1.020,1.0,0.0,121.000000,36.0,1.2,137.528754,4.627244,15.4
1,1.0,7.0,50.0,1.020,4.0,0.0,148.036517,18.0,0.8,137.528754,4.627244,11.3
2,2.0,62.0,80.0,1.010,2.0,3.0,423.000000,53.0,1.8,137.528754,4.627244,9.6
3,3.0,48.0,70.0,1.005,4.0,0.0,117.000000,56.0,3.8,111.000000,2.500000,11.2
4,4.0,51.0,80.0,1.010,2.0,0.0,106.000000,26.0,1.4,137.528754,4.627244,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...
395,395.0,55.0,80.0,1.020,0.0,0.0,140.000000,49.0,0.5,150.000000,4.900000,15.7
396,396.0,42.0,70.0,1.025,0.0,0.0,75.000000,31.0,1.2,141.000000,3.500000,16.5
397,397.0,12.0,80.0,1.020,0.0,0.0,100.000000,26.0,0.6,137.000000,4.400000,15.8
398,398.0,17.0,60.0,1.025,0.0,0.0,114.000000,50.0,1.0,135.000000,4.900000,14.2


In [13]:
dataset[qual]

,rbc,pc,pcc,ba,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,normal,normal,notpresent,notpresent,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,normal,normal,notpresent,notpresent,38,6000,5.2,no,no,no,good,no,no,ckd
2,normal,normal,notpresent,notpresent,31,7500,5.2,no,yes,no,poor,no,yes,ckd
3,normal,abnormal,present,notpresent,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,normal,normal,notpresent,notpresent,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,normal,normal,notpresent,notpresent,47,6700,4.9,no,no,no,good,no,no,notckd
396,normal,normal,notpresent,notpresent,54,7800,6.2,no,no,no,good,no,no,notckd
397,normal,normal,notpresent,notpresent,49,6600,5.4,no,no,no,good,no,no,notckd
398,normal,normal,notpresent,notpresent,51,7200,5.9,no,no,no,good,no,no,notckd


In [14]:
print("Remaining missing values:", dataset.isnull().sum())

Remaining missing values: id                0
age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64


## Concatenating Preprocessed Columns
- **Purpose:** Combine imputed numerical and categorical columns back into a unified DataFrame.

In [15]:
df = pd.DataFrame(dataset, columns=quan)
cate = pd.DataFrame(dataset, columns=qual)

In [16]:
df

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
0,0.0,48.0,80.0,1.020,1.0,0.0,121.000000,36.0,1.2,137.528754,4.627244,15.4
1,1.0,7.0,50.0,1.020,4.0,0.0,148.036517,18.0,0.8,137.528754,4.627244,11.3
2,2.0,62.0,80.0,1.010,2.0,3.0,423.000000,53.0,1.8,137.528754,4.627244,9.6
3,3.0,48.0,70.0,1.005,4.0,0.0,117.000000,56.0,3.8,111.000000,2.500000,11.2
4,4.0,51.0,80.0,1.010,2.0,0.0,106.000000,26.0,1.4,137.528754,4.627244,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...
395,395.0,55.0,80.0,1.020,0.0,0.0,140.000000,49.0,0.5,150.000000,4.900000,15.7
396,396.0,42.0,70.0,1.025,0.0,0.0,75.000000,31.0,1.2,141.000000,3.500000,16.5
397,397.0,12.0,80.0,1.020,0.0,0.0,100.000000,26.0,0.6,137.000000,4.400000,15.8
398,398.0,17.0,60.0,1.025,0.0,0.0,114.000000,50.0,1.0,135.000000,4.900000,14.2


In [17]:
cate

,rbc,pc,pcc,ba,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,normal,normal,notpresent,notpresent,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,normal,normal,notpresent,notpresent,38,6000,5.2,no,no,no,good,no,no,ckd
2,normal,normal,notpresent,notpresent,31,7500,5.2,no,yes,no,poor,no,yes,ckd
3,normal,abnormal,present,notpresent,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,normal,normal,notpresent,notpresent,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,normal,normal,notpresent,notpresent,47,6700,4.9,no,no,no,good,no,no,notckd
396,normal,normal,notpresent,notpresent,54,7800,6.2,no,no,no,good,no,no,notckd
397,normal,normal,notpresent,notpresent,49,6600,5.4,no,no,no,good,no,no,notckd
398,normal,normal,notpresent,notpresent,51,7200,5.9,no,no,no,good,no,no,notckd


In [18]:
preprocessed = pd.concat([df, cate], axis=1)
preprocessed

,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0.0,48.0,80.0,1.020,1.0,0.0,121.000000,36.0,1.2,137.528754,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1.0,7.0,50.0,1.020,4.0,0.0,148.036517,18.0,0.8,137.528754,...,38,6000,5.2,no,no,no,good,no,no,ckd
2,2.0,62.0,80.0,1.010,2.0,3.0,423.000000,53.0,1.8,137.528754,...,31,7500,5.2,no,yes,no,poor,no,yes,ckd
3,3.0,48.0,70.0,1.005,4.0,0.0,117.000000,56.0,3.8,111.000000,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4.0,51.0,80.0,1.010,2.0,0.0,106.000000,26.0,1.4,137.528754,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395.0,55.0,80.0,1.020,0.0,0.0,140.000000,49.0,0.5,150.000000,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396.0,42.0,70.0,1.025,0.0,0.0,75.000000,31.0,1.2,141.000000,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397.0,12.0,80.0,1.020,0.0,0.0,100.000000,26.0,0.6,137.000000,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398.0,17.0,60.0,1.025,0.0,0.0,114.000000,50.0,1.0,135.000000,...,51,7200,5.9,no,no,no,good,no,no,notckd


## Descriptive Statistics
## Statistical Summary
- **Purpose:** Generate descriptive statistics for numerical columns

In [19]:
dataset.describe()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,199.500000,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437
std,115.614301,16.974966,13.476298,0.005369,1.272318,1.029487,74.782634,49.285887,5.617490,9.204273,2.819783,2.716171
min,0.000000,2.000000,50.000000,1.005000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000
25%,99.750000,42.000000,70.000000,1.015000,0.000000,0.000000,101.000000,27.000000,0.900000,135.000000,4.000000,10.875000
50%,199.500000,54.000000,78.234536,1.017408,1.000000,0.000000,126.000000,44.000000,1.400000,137.528754,4.627244,12.526437
75%,299.250000,64.000000,80.000000,1.020000,2.000000,0.450142,150.000000,61.750000,3.072454,141.000000,4.800000,14.625000
max,399.000000,90.000000,180.000000,1.025000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000


## Detailed Statistics
- **Purpose:** Calculate measures like mean, median, mode, and IQR for each numerical column.

In [20]:
# Assuming 'quan' is defined somewhere in your code
descriptive = pd.DataFrame(index=['mean','median','mode','Q1:25%','Q2:50%','Q3:75%','Q4:100%','IQR','1.5rule','lesser','greater','min','max'], columns=quan)

# Assuming 'dataset' is your DataFrame
for ColumnName in quan:
    descriptive.loc['mean', ColumnName] = dataset[ColumnName].mean()
    descriptive.loc['median', ColumnName] = dataset[ColumnName].median()
    descriptive.loc['mode', ColumnName] = dataset[ColumnName].mode()[0]
    descriptive.loc['Q1:25%', ColumnName] = dataset[ColumnName].quantile(0.25)
    descriptive.loc['Q2:50%', ColumnName] = dataset[ColumnName].quantile(0.50)
    descriptive.loc['Q3:75%', ColumnName] = dataset[ColumnName].quantile(0.75)
    descriptive.loc['99%', ColumnName] = np.percentile(dataset[ColumnName], 99)
    descriptive.loc['Q4:100%', ColumnName] = dataset[ColumnName].max()
    descriptive.loc['IQR', ColumnName] = descriptive.loc['Q3:75%', ColumnName] - descriptive.loc['Q1:25%', ColumnName]
    descriptive.loc['1.5rule', ColumnName] = 1.5 * descriptive.loc['IQR', ColumnName]
    descriptive.loc['lesser', ColumnName] = descriptive.loc['Q1:25%', ColumnName] - descriptive.loc['1.5rule', ColumnName]
    descriptive.loc['greater', ColumnName] = descriptive.loc['Q3:75%', ColumnName] + descriptive.loc['1.5rule', ColumnName]
    descriptive.loc['min', ColumnName] = dataset[ColumnName].min()
    descriptive.loc['max', ColumnName] = dataset[ColumnName].max()

descriptive

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
mean,199.5,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437
median,199.5,54.0,78.234536,1.017408,1.0,0.0,126.0,44.0,1.4,137.528754,4.627244,12.526437
mode,0.0,60.0,80.0,1.02,0.0,0.0,148.036517,57.425722,1.2,137.528754,4.627244,12.526437
Q1:25%,99.75,42.0,70.0,1.015,0.0,0.0,101.0,27.0,0.9,135.0,4.0,10.875
Q2:50%,199.5,54.0,78.234536,1.017408,1.0,0.0,126.0,44.0,1.4,137.528754,4.627244,12.526437
Q3:75%,299.25,64.0,80.0,1.02,2.0,0.450142,150.0,61.75,3.072454,141.0,4.8,14.625
Q4:100%,399.0,90.0,180.0,1.025,5.0,5.0,490.0,391.0,76.0,163.0,47.0,17.8
IQR,199.5,22.0,10.0,0.005,2.0,0.450142,49.0,34.75,2.172454,6.0,0.8,3.75
1.5rule,299.25,33.0,15.0,0.0075,3.0,0.675214,73.5,52.125,3.258681,9.0,1.2,5.625
lesser,-199.5,9.0,55.0,1.0075,-3.0,-0.675214,27.5,-25.125,-2.358681,126.0,2.8,5.25


## Outlier Handling
## Identifying Outliers
- **Purpose:** Detect columns with outliers based on IQR rules.

In [21]:
lesser=[]
greater=[]

for columnName in quan:
    if(descriptive[columnName]["min"]<descriptive[columnName]["lesser"]):
        lesser.append(columnName)
    if(descriptive[columnName] ["max"]>descriptive[columnName]["greater"]):
        greater.append(columnName)

In [22]:
lesser

['age', 'bp', 'sg', 'bgr', 'sod', 'pot', 'hemo']

In [23]:
greater

['bp', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot']

In [24]:
# Applying the 'lesser' condition with .loc[]
for columnName in lesser:
    dataset.loc[dataset[columnName] < descriptive.loc['lesser', columnName], columnName] = descriptive.loc['lesser', columnName]

# Applying the 'greater' condition with .loc[]
for columnName in greater:
    dataset.loc[dataset[columnName] > descriptive.loc['greater', columnName], columnName] = descriptive.loc['greater', columnName]

In [25]:
# Assuming 'quan' is defined somewhere in your code
descriptive = pd.DataFrame(index=['mean','median','mode','Q1:25%','Q2:50%','Q3:75%','Q4:100%','IQR','1.5rule','lesser','greater','min','max'], columns=quan)

# Assuming 'dataset' is your DataFrame
for ColumnName in quan:
    descriptive.loc['mean', ColumnName] = dataset[ColumnName].mean()
    descriptive.loc['median', ColumnName] = dataset[ColumnName].median()
    descriptive.loc['mode', ColumnName] = dataset[ColumnName].mode()[0]
    descriptive.loc['Q1:25%', ColumnName] = dataset[ColumnName].quantile(0.25)
    descriptive.loc['Q2:50%', ColumnName] = dataset[ColumnName].quantile(0.50)
    descriptive.loc['Q3:75%', ColumnName] = dataset[ColumnName].quantile(0.75)
    descriptive.loc['99%', ColumnName] = np.percentile(dataset[ColumnName], 99)
    descriptive.loc['Q4:100%', ColumnName] = dataset[ColumnName].max()
    descriptive.loc['IQR', ColumnName] = descriptive.loc['Q3:75%', ColumnName] - descriptive.loc['Q1:25%', ColumnName]
    descriptive.loc['1.5rule', ColumnName] = 1.5 * descriptive.loc['IQR', ColumnName]
    descriptive.loc['lesser', ColumnName] = descriptive.loc['Q1:25%', ColumnName] - descriptive.loc['1.5rule', ColumnName]
    descriptive.loc['greater', ColumnName] = descriptive.loc['Q3:75%', ColumnName] + descriptive.loc['1.5rule', ColumnName]
    descriptive.loc['min', ColumnName] = dataset[ColumnName].min()
    descriptive.loc['max', ColumnName] = dataset[ColumnName].max()

In [26]:
descriptive

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
mean,199.5,51.568376,75.719072,1.017452,1.016949,0.222685,136.985267,51.010409,2.254129,138.115004,4.432744,12.532937
median,199.5,54.0,78.234536,1.017408,1.0,0.0,126.0,44.0,1.4,137.528754,4.627244,12.526437
mode,0.0,60.0,80.0,1.02,0.0,0.0,223.5,113.875,6.331136,137.528754,4.627244,12.526437
Q1:25%,99.75,42.0,70.0,1.015,0.0,0.0,101.0,27.0,0.9,135.0,4.0,10.875
Q2:50%,199.5,54.0,78.234536,1.017408,1.0,0.0,126.0,44.0,1.4,137.528754,4.627244,12.526437
Q3:75%,299.25,64.0,80.0,1.02,2.0,0.450142,150.0,61.75,3.072454,141.0,4.8,14.625
Q4:100%,399.0,90.0,95.0,1.025,5.0,1.125356,223.5,113.875,6.331136,150.0,6.0,17.8
IQR,199.5,22.0,10.0,0.005,2.0,0.450142,49.0,34.75,2.172454,6.0,0.8,3.75
1.5rule,299.25,33.0,15.0,0.0075,3.0,0.675214,73.5,52.125,3.258681,9.0,1.2,5.625
lesser,-199.5,9.0,55.0,1.0075,-3.0,-0.675214,27.5,-25.125,-2.358681,126.0,2.8,5.25


In [27]:
def find_outliers(quan):
    lesser = []
    greater = []
    
    for columnName in quan:
        if descriptive[columnName]["min"] < descriptive[columnName]["lesser"]:
            lesser.append(columnName)
        if descriptive[columnName]["max"] > descriptive[columnName]["greater"]:
            greater.append(columnName)
    
    return lesser, greater

In [28]:
find_outliers(quan)

([], [])

In [29]:
dataset.to_csv('cleaned_kidney_disease.csv', index=False)
print("Cleaned data saved to 'cleaned_kidney_disease.csv'.")


Cleaned data saved to 'cleaned_kidney_disease.csv'.
